The goal is train 

In [ ]:
# Import Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt

# Setup tensorflow GPUs
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

# precision
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Prepare Images

In [ ]:
# Load the MNIST Dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Print out an image of size 28x28
print(f'Label {train_labels[0]}')
digit = train_images[0]
plt.imshow( digit, cmap=plt.cm.binary )
plt.show()

In [ ]:
# Change images to smaller image size of 8x8
train_images1 = tf.expand_dims(train_images, axis=-1)
train_images2 = tf.image.resize(train_images1, [8,8])
train_images_8x8 = tf.squeeze(train_images2, axis=-1).numpy()

test_images1 = tf.expand_dims(test_images, axis=-1)
test_images2 = tf.image.resize(test_images1, [8,8])
test_images_8x8 = tf.squeeze(test_images2, axis=-1).numpy()

# Reshape 8x8 array to a one dimensional 1x64 so it can be used as the input layer for the neural network below
train_images = train_images_8x8.reshape((60000, 8 * 8))
train_images = train_images.astype("float32") / 255
test_images = test_images_8x8.reshape((10000, 8 * 8))
test_images = test_images.astype("float32") / 255

# Verify new training shape of (60000, 8, 8)
train_images_8x8.shape

In [ ]:
# Print out an image of size 8x8. Notice the smaller image size than above.
print(f'Label {train_labels[0]}')
digit = train_images_8x8[0]
plt.imshow( digit, cmap=plt.cm.binary )
plt.show()

# Prepare Model

In [ ]:
# Create a custom constraint function so that weights will be constraint to positive values.
from keras.constraints import Constraint
import keras.backend as K

class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, w):        
        return K.clip(w, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}

In [ ]:
# Model
# Set bias term to false
model = keras.Sequential( [
    layers.Dense( 10, activation='relu', kernel_constraint=Between(min_value=0, max_value=100), use_bias=False ),
    layers.Dense( 10, activation='softmax', kernel_constraint=Between(min_value=0, max_value=100), use_bias=False )
    ] )

# Training specifictations
model.compile(  optimizer='rmsprop',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'] )

In [ ]:
# Fit model
# Around 100 epochs are enough to reach an adequate accuracy for the given image and neural net size. 
model.fit(train_images, train_labels, epochs=100, batch_size=128)
model.summary()

In [ ]:
# Check accuracy on the test set.
test_loss, test_acc = model.evaluate(test_images, test_labels)
print( f'Test Accuracy: {test_acc}' )

In [ ]:
# Save weights
weights_list = []
for layer in model.layers:
    weights = layer.get_weights()
    weights_list.append(weights[0].tolist())


with open(r'weights.txt', 'w') as fp:
    fp.write(str(weights_list))

In [ ]:
# Check if any of the elements in the array are negative
my_array = np.array(weights_list[0])

if (my_array < 0).any():
    print("The array contains negative numbers.")
else:
    print("The array does not contain any negative numbers.")


# Export lower resolution image  

In [ ]:
# Export the first 20 images of the test set as one dimensional arrays.
l2 = []
export = []
for i in range(len(test_labels[:20])):
    # print("let image_"+str(i)+"_label_"+str(Y_test[i])+" = "+str(X_test[i].reshape(-1).astype("float32")/255))
    l2.append(f"let image_{i}_label_{test_labels[i]} = {list(test_images[i].reshape(-1).astype('float32')/255)}")
    export.append(f"image_{i}_label_{test_labels[i]}")

In [ ]:
with open(r'test_images_8x8.txt', 'w') as fp:
    fp.write(str(export))
    fp.write("\n")
    for item in l2:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')